#### References:<br>
https://docs.azuredatabricks.net/user-guide/importing-data.html<br>
https://docs.azuredatabricks.net/spark/latest/faq/join-two-dataframes-duplicated-column.html<br>

https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame<br>

In [2]:
from pyspark.sql.types import *

In [3]:
source_file = "/mnt/hack/csv/sample/dat202/airports.csv"

#### With header, and schema inference

In [5]:
df_inferred = spark\
    .read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(source_file)

#### With header, and explicit schema

In [7]:
# Define the schema explicitly

data_schema = StructType([
  StructField("airport_id", IntegerType(), True),
  StructField("city", StringType(), True),
  StructField("state", StringType(), True),
  StructField("name", StringType(), True)
])

In [8]:
# Read the data into a dataframe, explicitly specifying the schema we defined above

df_explicit = spark\
    .read\
    .format("csv")\
    .option("header", "true")\
    .schema(data_schema)\
    .load(source_file)